<a href="https://colab.research.google.com/github/RaminVali/Island_Project/blob/main/scraping.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Setting up Selenium Chorme dirver to use for scraping and scrolling donwn etc.

Google colab has issues with selenium and the associated websidrver because their ubunto release does not include snap. Here is a workaround for this issue on google colab. Please see [here](https://stackoverflow.com/questions/75164313/selenium-in-google-colab-stopped-working-showing-an-error-as-service-chromedrive) for more details.

In [14]:
%%shell
# Ubuntu no longer distributes chromium-browser outside of snap
#
# Proposed solution: https://askubuntu.com/questions/1204571/how-to-install-chromium-without-snap

# Add debian buster
cat > /etc/apt/sources.list.d/debian.list <<'EOF'
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-buster.gpg] http://deb.debian.org/debian buster main
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-buster-updates.gpg] http://deb.debian.org/debian buster-updates main
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-security-buster.gpg] http://deb.debian.org/debian-security buster/updates main
EOF

# Add keys
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys DCC9EFBF77E11517
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys 648ACFD622F3D138
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys 112695A0E562B32A

apt-key export 77E11517 | gpg --dearmour -o /usr/share/keyrings/debian-buster.gpg
apt-key export 22F3D138 | gpg --dearmour -o /usr/share/keyrings/debian-buster-updates.gpg
apt-key export E562B32A | gpg --dearmour -o /usr/share/keyrings/debian-security-buster.gpg

# Prefer debian repo for chromium* packages only
# Note the double-blank lines between entries
cat > /etc/apt/preferences.d/chromium.pref << 'EOF'
Package: *
Pin: release a=eoan
Pin-Priority: 500


Package: *
Pin: origin "deb.debian.org"
Pin-Priority: 300


Package: chromium*
Pin: origin "deb.debian.org"
Pin-Priority: 700
EOF

# Install chromium and chromium-driver
apt-get install chromium chromium-driver
apt-get update


# Install selenium
pip install selenium

# Install
pip install webdriver-manager

Executing: /tmp/apt-key-gpghome.4DtJhP0DK5/gpg.1.sh --keyserver keyserver.ubuntu.com --recv-keys DCC9EFBF77E11517
gpg: key DCC9EFBF77E11517: "Debian Stable Release Key (10/buster) <debian-release@lists.debian.org>" not changed
gpg: Total number processed: 1
gpg:              unchanged: 1
Executing: /tmp/apt-key-gpghome.3QYnBn0Xzs/gpg.1.sh --keyserver keyserver.ubuntu.com --recv-keys 648ACFD622F3D138
gpg: key DC30D7C23CBBABEE: "Debian Archive Automatic Signing Key (10/buster) <ftpmaster@debian.org>" not changed
gpg: Total number processed: 1
gpg:              unchanged: 1
Executing: /tmp/apt-key-gpghome.Taoa53L5E3/gpg.1.sh --keyserver keyserver.ubuntu.com --recv-keys 112695A0E562B32A
gpg: key 4DFAB270CAA96DFA: "Debian Security Archive Automatic Signing Key (10/buster) <ftpmaster@debian.org>" not changed
gpg: Total number processed: 1
gpg:              unchanged: 1
gpg: cannot open '/dev/tty': No such device or address
gpg: [stdout]: write error: Broken pipe
gpg: filter_flush failed on c

In [15]:
! ls /usr/bin/chromedriver

/usr/bin/chromedriver


Importing Packages

In [16]:
import time
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.common.by import By

import os
from bs4 import BeautifulSoup
from urllib.parse import urljoin
from collections import defaultdict

import re


import requests
import pandas as pd

# Function Declerations

## Setting up the webdriver with appropriate options
### Setting up a function for the driver so we always inherit the options
To not be flagged as a robot by the website, see [here](https://stackoverflow.com/questions/71885891/urllib3-exceptions-maxretryerror-httpconnectionpoolhost-localhost-port-5958)

In [13]:
def get_driver():
  #service = Service(executable_path='/usr/lib/chromium-browser/chromedriver/usr/bin') #this gives a version mismatch error bweteen the driver and the Chormium
  service = Service(executable_path='/usr/bin/chromedriver') #this is where the correct version of the chromium and webdriver live
  options = Options()
  options.add_argument("--headless")
  options.add_argument("--no-sandbox")
  options.add_argument('--disable-dev-shm-usage')
  options.add_argument('--disable-blink-features=AutomationControlled')
  driver = webdriver.Chrome(service = service, options = options)
  return driver


# function to manipulate a list into two lists
def zigzag(seq):
  results = [], []
  for i, e in enumerate(seq):
    results[i%2].append(e)
  return results


Setting up the driver and doing infinite scrolling to get all the html form the main website page. We will get the hrefs with all the links. [Source](https://medium.com/analytics-vidhya/using-python-and-selenium-to-scrape-infinite-scroll-web-pages-825d12c24ec7)

Only uncomment the below if you want the main page of the ```privateislands.com``` to be scanned. I have already scanned this file and I use it for all the testing. It is saved as output1.

In [5]:
# # Web scrapper for infinite scrolling page
# driver = get_driver()
# driver.get("https://www.privateislandsonline.com/search?availability=sale&region=&q2=&price_range=0%3A50000000&size_range=0%3A1000")
# time.sleep(2)  # Allow 2 seconds for the web page to open
# scroll_pause_time = 2 # You can set your own pause time. My laptop is a bit slow so I use 1 sec
# screen_height = driver.execute_script("return window.screen.height;")   # get the screen height of the web
# i = 1

# while True:
#     # scroll one screen height each time
#     driver.execute_script("window.scrollTo(0, {screen_height}*{i});".format(screen_height=screen_height, i=i))
#     i += 1
#     time.sleep(scroll_pause_time)
#     # update scroll height each time after scrolled, as the scroll height can change after we scrolled the page
#     scroll_height = driver.execute_script("return document.body.scrollHeight;")
#     # Break the loop when the height we need to scroll to is larger than the total scroll height
#     if (screen_height) * i > scroll_height:
#         break

# soup = BeautifulSoup(driver.page_source, "html.parser")


Saving the contents we scraped. Uncomment as above.

In [6]:
# with open("output1.html", "w", encoding='utf-8') as file:
#     file.write(str(soup))

Loading the saved contents again

In [7]:
path = '/content/drive/MyDrive/Colab Notebooks/Private_Island_Project/Island_Project/output1.html'

with open(path, 'r') as f:
    contents = f.read()
    soup = BeautifulSoup(contents, 'lxml')

##### Extract Individual Island URLs #####
urls = []
for parent in soup.find_all('div', class_="img-container"):
    url = parent.find('a').get('href')
    urls.append(url)

print(f"We have found {len(urls)} islands for sale")

We have found 611 islands for sale


We will go through each 611 URLs and dump the page contents into a seperate folder. The folder name is going to be from the URL, where it is region/country/island_name or region/region/country/island_name.

test_url = ['https://www.privateislandsonline.com/asia/thailand/rangyai-island',
 'https://www.privateislandsonline.com/europe/spain/2867']

In [8]:
test_url = urls[0]
test_url

'https://www.privateislandsonline.com/asia/thailand/rangyai-island'

In [9]:
test_url = ['https://www.privateislandsonline.com/europe/spain/2867','https://www.privateislandsonline.com/asia/thailand/rangyai-island']

In [10]:
DATA_DIR = "/content/drive/MyDrive/Colab Notebooks/Private_Island_Project/Island_Project/Data"


In [18]:
output = pd.DataFrame()
Driver = get_driver()
counter = 0

for url in test_url:

  Driver.get(url)
  print(f"index {counter}/611; title is: {Driver.title}")

  time.sleep(5)


  # extracting the text part of islands-details section
  basic_info = Driver.find_element(By.ID,"island-details")

  # cleaning the extracted text
  to_delete = ['FOR SALE','EUR','GBP','AUD','CAD','CZK','DKK','HKD','HUF','ILS',
              'JPY','MXN','NOK','NZD','PHP','PLN','SGD','SEK','CHF','TWD','THB',
              'UAH','ISK','HRK','RON','BGN','TRY','CLP','ZAR','BRL','MYR','RUB',
              'IDR','INR','KRW','CNY', 'USD','Back','Print','What is a blackbook Listing?',
              'Blackbook private island listings are available to the public, without','For Sale']
  basic_text = basic_info.text
  clean_text = re.sub(r'(\n\n\n\n\n\n\n\n|\n\n)', '', basic_text)
  lines = clean_text.split("\n")
  clean_list = [line for line in lines if not any(sub_delete in line for sub_delete in to_delete)]

  # Splitting the created list into dictionary key values
  keys, values = zigzag(clean_list)

  #print(clean_text)
  # print(keys)
  # print('\n')
  # print(values)

  [v.replace("[","").replace("]","") for v in values]
  #print(values)
  keys[-1] = 'Details'
  # creating a dictionary of the results
  res = defaultdict(list)
  for i, j in zip(keys, values):
    res[i].append(j)
  dict(res)

  # appending dictionary to dataframe

  df_dictionary = pd.DataFrame(res)
  output = pd.concat([output, df_dictionary], ignore_index=True)
  #output.head()
  counter +=1

#Driver.quit()


index 0/611; title is: 2867 - Spain, Europe - Private Islands for Sale
index 1/611; title is: Rangyai Island - Thailand, Asia - Private Islands for Sale


In [19]:
output

,Name,Region,Development,Title,Type,Price,Size,Lifestyles,Details,Location
0,Blackbook Listing (ID 2867),"Spain , Europe",Developed,Freehold,Private Island,"$100,000,000 - $200,000,000",100 - 200 Acres,"Large Acreage, Ocean Island, Turn Key, Income ...",Large turnkey private island available for sal...,NaN
1,Rangyai Island,"Thailand , Asia",Developed,Freehold,Private Island,"$ 160,000,000",110.00 Acres,"Islands With Beaches, Large Acreage, Ocean Isl...","Located just east of the island of Phuket, an ...",East of Phuket


In [ ]:
imgs = Driver.find_elements("xpath","//img")
img_list = []
for img in imgs:
    img_list.append(img.get_attribute("src"))

gen = (
    l for l in img_list
    if '.jpeg' in l
    )



NameError: ignored

In [ ]:
next(gen)

StopIteration: ignored

In [ ]:
l

NameError: ignored

In [ ]:
img_list

['https://www.privateislandsonline.com/img/front/new-logo.png',
 'https://www.privateislandsonline.com/img/front/logo_small.png',
 'https://www.privateislandsonline.com/img/front/navSocialIcon_fb.png',
 'https://www.privateislandsonline.com/img/front/navSocialIcon_tw.png',
 'https://www.privateislandsonline.com/img/front/navSocialIcon_gp.png',
 'https://www.google.com/images/cleardot.gif',
 'https://www.google.com/images/cleardot.gif',
 'https://www.google.com/images/cleardot.gif',
 'https://www.privateislandsonline.com/images/mappoint.png',
 'https://www.privateislandsonline.com/uploads/resize/_597_image_fe02c338d8.jpg-1360-1100.jpg',
 'https://www.privateislandsonline.com/uploads/resize/_597_image_25bce99231.jpg-1360-1100.jpg',
 'https://www.privateislandsonline.com/uploads/resize/_597_image_d24a8525fc.jpg-1360-1100.jpg',
 'https://www.privateislandsonline.com/uploads/resize/_597_image_e971b277e1.jpg-1360-1100.jpg',
 'https://www.privateislandsonline.com/uploads/resize/_597_image_fe0

In [ ]:
gen

<generator object <genexpr> at 0x7d0261401690>

In [ ]:
img = Driver.find_elements("xpath","//img[@ class = 'owl-item.cloned']")

In [ ]:
img

[]

In [ ]:
imgs

[]